### Importing Libraries and Source Functions

In [1]:
import sys
import os

from src.python import check_and_pull_git_lfs, read_large_file, pivot_dfs_smart, set_columns_to_datetime, \
    filter_rows_by_value, remove_duplicate_rows, detailed_duplicate_analysis, describe_dataframes, \
    save_df_list_to_csv_auto, coalesce_columns, remove_columns

sys.path.append(os.path.abspath(os.path.join('..', '..', 'src', 'python')))
from Functions import *


### Loading the Dataset from CSV

In [2]:
#check_and_pull_git_lfs() # Source function to check and pull git-lfs files
# Be careful if directory is not set it will pull all files in the repository, a lot of data!

# Load the dataset from CSV using source function, can use bonus auto_handle_lfs=True parameter to pull the needed git-lfs file.
balance2025 = read_large_file('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2025_n.csv')
balance2024 = read_large_file('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2024_n.csv')
balance2023 = read_large_file('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2023.csv')
balance2022 = read_large_file('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2022.csv')
balance2021 = read_large_file('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2021.csv')
balance2020 = read_large_file('../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2020.csv')

pnl_2025 = read_large_file('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2025_n.csv')
pnl_2024 = read_large_file('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2024_n.csv')
pnl_2023 = read_large_file('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2023.csv')
pnl_2022 = read_large_file('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2022.csv')
pnl_2021 = read_large_file('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2021.csv')
pnl_2020 = read_large_file('../../data/raw/firm-PnL-statements/JAR_FA_RODIKLIAI_PLNA_2020.csv')


Reading CSV file from: ../../data/raw/firm-balance-statements/JAR_FA_RODIKLIAI_BLNS_2025_n.csv
File size: 176.02 MB
Auto-detecting CSV delimiter...
Delimiter detection scores: {',': 1.125, '|': 16.0}
Detected delimiter: '|' (score: 16.00)
Bad lines handling: error
File encoding: utf-8
Processed chunk 1 with 10000 rows (total: 10000)
Processed chunk 2 with 10000 rows (total: 20000)
Processed chunk 3 with 10000 rows (total: 30000)
Processed chunk 4 with 10000 rows (total: 40000)
Processed chunk 5 with 10000 rows (total: 50000)
Processed chunk 6 with 10000 rows (total: 60000)
Processed chunk 7 with 10000 rows (total: 70000)
Processed chunk 8 with 10000 rows (total: 80000)
Processed chunk 9 with 10000 rows (total: 90000)
Processed chunk 10 with 10000 rows (total: 100000)
Processed chunk 11 with 10000 rows (total: 110000)
Processed chunk 12 with 10000 rows (total: 120000)
Processed chunk 13 with 10000 rows (total: 130000)
Processed chunk 14 with 10000 rows (total: 140000)
Processed chunk 15

### Making backups of the original datasets

In [3]:
# Balance statements
B2025 = balance2025.copy()
B2024 = balance2024.copy()
B2023 = balance2023.copy()
B2022 = balance2022.copy()
B2021 = balance2021.copy()
B2020 = balance2020.copy()

# PnL statements
P2025 = pnl_2025.copy()
P2024 = pnl_2024.copy()
P2023 = pnl_2023.copy()
P2022 = pnl_2022.copy()
P2021 = pnl_2021.copy()
P2020 = pnl_2020.copy()

# Saving into list for further processing
balance_statements = [B2025, B2024, B2023, B2022, B2021, B2020]
pnl_statements = [P2025, P2024, P2023, P2022, P2021, P2020]

### Removing unnecessary columns

In [4]:
# Balance statements

bad_cols_bal = ['obj_pav', 'form_pav', 'stat_pav', 'template_name', 'standard_name', 'formavimo_data', 'ja_pavadinimas', 'form_pavadinimas', 'stat_pavadinimas', 'line_type_id' ]

balance_statements = remove_columns(balance_statements, bad_cols_bal, verbose=True, inplace=True)

# PnL statements

bad_cols_pnl = ['obj_pav', 'form_pav', 'stat_pav', 'template_name', 'standard_name', 'formavimo_data', 'ja_pavadinimas', 'form_pavadinimas', 'stat_pavadinimas', 'line_type_id' ]

pnl_statements = remove_columns(pnl_statements, bad_cols_pnl, verbose=True, inplace=True)

📊 DataFrame 0:
   ✅ Removed: ['template_name', 'standard_name', 'formavimo_data', 'ja_pavadinimas', 'form_pavadinimas', 'stat_pavadinimas', 'line_type_id']
   ⏭️  Skipped: ['obj_pav', 'form_pav', 'stat_pav']
   📋 Remaining columns: 10
📊 DataFrame 1:
   ✅ Removed: ['template_name', 'standard_name', 'formavimo_data', 'ja_pavadinimas', 'form_pavadinimas', 'stat_pavadinimas', 'line_type_id']
   ⏭️  Skipped: ['obj_pav', 'form_pav', 'stat_pav']
   📋 Remaining columns: 10
📊 DataFrame 2:
   ✅ Removed: ['obj_pav', 'template_name', 'standard_name', 'formavimo_data']
   ⏭️  Skipped: ['form_pav', 'stat_pav', 'ja_pavadinimas', 'form_pavadinimas', 'stat_pavadinimas', 'line_type_id']
   📋 Remaining columns: 12
📊 DataFrame 3:
   ✅ Removed: ['obj_pav', 'form_pav', 'stat_pav', 'template_name', 'standard_name', 'formavimo_data']
   ⏭️  Skipped: ['ja_pavadinimas', 'form_pavadinimas', 'stat_pavadinimas', 'line_type_id']
   📋 Remaining columns: 12
📊 DataFrame 4:
   ✅ Removed: ['obj_pav', 'form_pav', 'stat_p

### Renaming columns to be the same across all datasets

In [5]:
# Balance statements
# General columns
column_list = ['obj_kodas', 'stat_statusas', 'laikotarpis_nuo', 'laikotarpis_iki']
rename_list = ['ja_kodas', 'stat_kodas','beginning_date', 'turning_date']

# Financial columns
financial_cols_bal = ['nuosavas_kapitalas', 'mok_sumos_ir_isipareigojimai', 'ilgalaikis_turtas', 'trumpalaikis_turtas']
financial_rename_bal = ['NUOSAVAS KAPITALAS','MOKĖTINOS SUMOS IR KITI ĮSIPAREIGOJIMAI', 'ILGALAIKIS TURTAS', 'TRUMPALAIKIS TURTAS']

# Rename columns
balance_statements = rename_columns_if_exist(balance_statements, column_list, rename_list)
balance_statements = rename_columns_if_exist(balance_statements, financial_cols_bal, financial_rename_bal)

# PnL statements

#General columns same as balance statements

# Financial columns
financial_cols_pnl = ['pelnas_pries_apmokestinima', 'grynasis_pelnas', 'pardavimo_pajamos']
financial_rename_pnl = ['PELNAS (NUOSTOLIAI) PRIEŠ APMOKESTINIMĄ', 'GRYNASIS PELNAS (NUOSTOLIAI)', 'PARDAVIMO PAJAMOS']

# Rename columns
pnl_statements = rename_columns_if_exist(pnl_statements, column_list, rename_list)
pnl_statements = rename_columns_if_exist(pnl_statements, financial_cols_pnl, financial_rename_pnl)

### Pivoting the data for new columns

In [6]:
# Balance statements

pivot_dfs_smart(balance_statements, date_column='reg_date', inplace=True)

# PnL statements

pivot_dfs_smart(pnl_statements, date_column='reg_date', inplace=True)

⚠️  INPLACE MODE: Modifying original DataFrame(s)
DataFrame 0: Preserving 8 columns in result


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



DataFrame 0: Successfully pivoted INPLACE. New shape: (150021, 17)
Removed 19853 duplicate rows
DataFrame 1: Preserving 8 columns in result


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



DataFrame 1: Successfully pivoted INPLACE. New shape: (145274, 17)
Removed 33091 duplicate rows
DataFrame 2: Missing columns ['line_name', 'reiksme']
DataFrame 3: Missing columns ['line_name', 'reiksme']
DataFrame 4: Missing columns ['line_name', 'reiksme']
DataFrame 5: Missing columns ['line_name', 'reiksme']

🎯 FINAL DUPLICATE REMOVAL SUMMARY (INPLACE):
   📊 Total duplicate rows removed across all DataFrames: 52944
   📅 Using date column 'reg_date' - kept most recent rows in duplicate groups
⚠️  INPLACE MODE: Modifying original DataFrame(s)
DataFrame 0: Preserving 8 columns in result

Exact Duplicate Detection Report:
Found 11906 duplicate groups (14947 total duplicate rows)
Duplicate definition: exact match on ja_kodas + line_name with differing reiksme
Using date column 'reg_date' - keeping most recent row in each duplicate group

Duplicate Group: ja_kodas=110053842, line_name=GRYNASIS PELNAS (NUOSTOLIAI)
 - Keeping most recent row (index 77)
 - Removing 1 duplicates (indices: [72]

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



DataFrame 1: Successfully pivoted INPLACE. New shape: (144312, 16)
Removed 23856 duplicate rows
DataFrame 2: Missing columns ['line_name', 'reiksme']
DataFrame 3: Missing columns ['line_name', 'reiksme']
DataFrame 4: Missing columns ['line_name', 'reiksme']
DataFrame 5: Missing columns ['line_name', 'reiksme']

🎯 FINAL DUPLICATE REMOVAL SUMMARY (INPLACE):
   📊 Total duplicate rows removed across all DataFrames: 38803
   📅 Using date column 'reg_date' - kept most recent rows in duplicate groups


### Making some collumns datetime

In [7]:
# All dfs share the same collumns

cols_to_datetime = ['reg_date', 'beginning_date', 'turning_date']

# Balance statements

set_columns_to_datetime(balance_statements, cols_to_datetime, inplace=True)

# PnL statements

set_columns_to_datetime(pnl_statements, cols_to_datetime, inplace=True)

⚠️  INPLACE MODE: Modifying original DataFrame(s)

Processing DataFrame 0 in place
Converted column 'reg_date' to datetime
Converted column 'beginning_date' to datetime
Converted column 'turning_date' to datetime
📊 Inplace processing: Converted 3, Skipped 0, Dropped 0

Processing DataFrame 1 in place
Converted column 'reg_date' to datetime
Converted column 'beginning_date' to datetime
Converted column 'turning_date' to datetime
📊 Inplace processing: Converted 3, Skipped 0, Dropped 0

Processing DataFrame 2 in place
Converted column 'reg_date' to datetime
Converted column 'beginning_date' to datetime
Converted column 'turning_date' to datetime
📊 Inplace processing: Converted 3, Skipped 0, Dropped 0

Processing DataFrame 3 in place
Converted column 'reg_date' to datetime
Converted column 'beginning_date' to datetime
Converted column 'turning_date' to datetime
📊 Inplace processing: Converted 3, Skipped 0, Dropped 0

Processing DataFrame 4 in place
Converted column 'reg_date' to datetime
C

### Removing wrong financial year data

In [8]:
# Balance statements

# Inclusive of new companies starting businesses after the start of the financial year
B2025_all = filter_rows_by_value(
    data= B2025,
    column={
        'beginning_date': ('2024-01-01', '2024-12-31'),
        'turning_date': ('2024-01-01', '2024-12-31'),
    },
    verbose=True
    )

B2024_all = filter_rows_by_value(
    data= B2024,
    column={'beginning_date': ('2023-01-01', '2023-12-31'), 'turning_date': ('2023-01-01', '2023-12-31')},
    verbose=True
    )

B2023_all = filter_rows_by_value(
    data= B2023,
    column={'beginning_date': ('2022-01-01', '2022-12-31'), 'turning_date': ('2022-01-01', '2022-12-31')},
    verbose=True
    )

B2022_all = filter_rows_by_value(
    data= B2022,
    column={'beginning_date': ('2021-01-01', '2021-12-31'), 'turning_date': ('2021-01-01', '2021-12-31')},
    verbose=True
    )

B2021_all = filter_rows_by_value(
    data= B2021,
    column={'beginning_date': ('2020-01-01', '2020-12-31'), 'turning_date': ('2020-01-01', '2020-12-31')},
    verbose=True
    )

B2020_all = filter_rows_by_value(
    data= B2020,
    column={'beginning_date': ('2019-01-01', '2019-12-31'), 'turning_date': ('2019-01-01', '2019-12-31')},
    verbose=True
    )

# Saving in list for further processing
balance_statements_all = [B2025_all, B2024_all, B2023_all, B2022_all, B2021_all, B2020_all]


# Only full financial year data

B2025_full = filter_rows_by_value(
    data= B2025,
    column={'beginning_date': ['2024-01-01'],
            'turning_date': ['2024-12-31']},
    verbose=True
    )

B2024_full = filter_rows_by_value(
    data= B2024,
    column={'beginning_date': ['2023-01-01'],
            'turning_date': ['2023-12-31']},
    verbose=True
    )

B2023_full = filter_rows_by_value(
    data= B2023,
    column={'beginning_date': ['2022-01-01'],
            'turning_date': ['2022-12-31']},
    verbose=True
    )

B2022_full = filter_rows_by_value(
    data= B2022,
    column={'beginning_date': ['2020-01-01'],
            'turning_date': ['2020-12-31']},
    verbose=True
    )

B2021_full = filter_rows_by_value(
    data= B2021,
    column={'beginning_date': ['2020-01-01'],
            'turning_date': ['2020-12-31']},
    verbose=True
    )
B2020_full = filter_rows_by_value(
    data= B2020,
    column={'beginning_date': ['2019-01-01'],
            'turning_date': ['2019-12-31']},
    verbose=True
    )

# Saving in list for further processing
balance_statements_full = [B2025_full, B2024_full, B2023_full, B2022_full, B2021_full, B2020_full]


🔍 Filter Conditions:
   - beginning_date: between 2024-01-01 and 2024-12-31
   - turning_date: between 2024-01-01 and 2024-12-31
🆕 NEW OBJECT MODE: Creating new DataFrame(s)
DataFrame 0: Original rows: 150021, Filtered rows: 143474, Removed: 6547

🎯 FILTERING SUMMARY:
   📊 Total rows removed across all DataFrames: 6547
🔍 Filter Conditions:
   - beginning_date: between 2023-01-01 and 2023-12-31
   - turning_date: between 2023-01-01 and 2023-12-31
🆕 NEW OBJECT MODE: Creating new DataFrame(s)
DataFrame 0: Original rows: 145274, Filtered rows: 137911, Removed: 7363

🎯 FILTERING SUMMARY:
   📊 Total rows removed across all DataFrames: 7363
🔍 Filter Conditions:
   - beginning_date: between 2022-01-01 and 2022-12-31
   - turning_date: between 2022-01-01 and 2022-12-31
🆕 NEW OBJECT MODE: Creating new DataFrame(s)
DataFrame 0: Original rows: 304469, Filtered rows: 136580, Removed: 167889

🎯 FILTERING SUMMARY:
   📊 Total rows removed across all DataFrames: 167889
🔍 Filter Conditions:
   - beginni

/home/aidmantas/repos/Lithuanian-Innovation-Agency-Risk-Model/src/python/Functions.py:2929: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  mask &= df[col].isin(condition['values'])


In [9]:
# PnL statements

# Inclusive of new companies starting businesses after the start of the financial year
P2025_all = filter_rows_by_value(
    data= P2025,
    column={
        'beginning_date': ('2024-01-01', '2024-12-31'),
        'turning_date': ('2024-01-01', '2024-12-31'),
    },
    verbose=True
    )

P2024_all = filter_rows_by_value(
    data= P2024,
    column={'beginning_date': ('2023-01-01', '2023-12-31'), 'turning_date': ('2023-01-01', '2023-12-31')},
    verbose=True
    )

P2023_all = filter_rows_by_value(
    data= P2023,
    column={'beginning_date': ('2022-01-01', '2022-12-31'), 'turning_date': ('2022-01-01', '2022-12-31')},
    verbose=True
    )

P2022_all = filter_rows_by_value(
    data= P2022,
    column={'beginning_date': ('2021-01-01', '2021-12-31'), 'turning_date': ('2021-01-01', '2021-12-31')},
    verbose=True
    )

P2021_all = filter_rows_by_value(
    data= P2021,
    column={'beginning_date': ('2020-01-01', '2020-12-31'), 'turning_date': ('2020-01-01', '2020-12-31')},
    verbose=True
    )

P2020_all = filter_rows_by_value(
    data= P2020,
    column={'beginning_date': ('2019-01-01', '2019-12-31'), 'turning_date': ('2019-01-01', '2019-12-31')},
    verbose=True
    )

# Saving in list for further processing
pnl_statements_all = [P2025_all, P2024_all, P2023_all, P2022_all, P2021_all, P2020_all]

# Only full financial year data

P2025_full = filter_rows_by_value(
    data= P2025,
    column={'beginning_date': ['2024-01-01'],
            'turning_date': ['2024-12-31']},
    verbose=True
    )

P2024_full = filter_rows_by_value(
    data= P2024,
    column={'beginning_date': ['2023-01-01'],
            'turning_date': ['2023-12-31']},
    verbose=True
    )

P2023_full = filter_rows_by_value(
    data= P2023,
    column={'beginning_date': ['2022-01-01'],
            'turning_date': ['2022-12-31']},
    verbose=True
    )

P2022_full = filter_rows_by_value(
    data= P2022,
    column={'beginning_date': ['2020-01-01'],
            'turning_date': ['2020-12-31']},
    verbose=True
    )

P2021_full = filter_rows_by_value(
    data= P2021,
    column={'beginning_date': ['2020-01-01'],
            'turning_date': ['2020-12-31']},
    verbose=True
    )

P2020_full = filter_rows_by_value(
    data= P2020,
    column={'beginning_date': ['2019-01-01'],
            'turning_date': ['2019-12-31']},
    verbose=True
    )

# Saving in list for further processing
pnl_statements_full = [P2025_full, P2024_full, P2023_full, P2022_full, P2021_full, P2020_full]

🔍 Filter Conditions:
   - beginning_date: between 2024-01-01 and 2024-12-31
   - turning_date: between 2024-01-01 and 2024-12-31
🆕 NEW OBJECT MODE: Creating new DataFrame(s)
DataFrame 0: Original rows: 149234, Filtered rows: 143206, Removed: 6028

🎯 FILTERING SUMMARY:
   📊 Total rows removed across all DataFrames: 6028
🔍 Filter Conditions:
   - beginning_date: between 2023-01-01 and 2023-12-31
   - turning_date: between 2023-01-01 and 2023-12-31
🆕 NEW OBJECT MODE: Creating new DataFrame(s)
DataFrame 0: Original rows: 144312, Filtered rows: 137499, Removed: 6813

🎯 FILTERING SUMMARY:
   📊 Total rows removed across all DataFrames: 6813
🔍 Filter Conditions:
   - beginning_date: between 2022-01-01 and 2022-12-31
   - turning_date: between 2022-01-01 and 2022-12-31
🆕 NEW OBJECT MODE: Creating new DataFrame(s)
DataFrame 0: Original rows: 141944, Filtered rows: 115654, Removed: 26290

🎯 FILTERING SUMMARY:
   📊 Total rows removed across all DataFrames: 26290
🔍 Filter Conditions:
   - beginning

### Removing older duplicates if they are present

In [10]:
# Balance statements

remove_duplicate_rows(balance_statements_all, 'ja_kodas',verbose=True,
date_column='reg_date', inplace=True)

remove_duplicate_rows(balance_statements_full, 'ja_kodas',verbose=True,
date_column='reg_date', inplace=True)

# PnL statements

remove_duplicate_rows(pnl_statements_all, 'ja_kodas',verbose=True, date_column='reg_date', inplace=True)

remove_duplicate_rows(pnl_statements_full, 'ja_kodas',verbose=True, date_column='reg_date', inplace=True)

🔍 Duplicate Removal Configuration:
   📊 Duplicate columns: ['ja_kodas']
   🎯 Keep method: first
   📅 Date column: reg_date (keeping most recent)
   💾 Inplace mode: True
⚠️  INPLACE MODE: Modifying original DataFrame(s)
DataFrame 0: Modified INPLACE. Original rows: 143474, Current rows: 143388, Removed: 86
DataFrame 1: Modified INPLACE. Original rows: 137911, Current rows: 137831, Removed: 80
DataFrame 2: Modified INPLACE. Original rows: 136580, Current rows: 136576, Removed: 4
DataFrame 3: Modified INPLACE. Original rows: 116772, Current rows: 116768, Removed: 4
DataFrame 4: Modified INPLACE. Original rows: 103750, Current rows: 103744, Removed: 6
DataFrame 5: Modified INPLACE. Original rows: 94666, Current rows: 94664, Removed: 2

🎯 DUPLICATE REMOVAL SUMMARY (INPLACE):
   📊 Total duplicate rows removed: 182
   🔍 Total duplicate groups found: 182
   📅 Using date column 'reg_date' - kept most recent rows
🔍 Duplicate Removal Configuration:
   📊 Duplicate columns: ['ja_kodas']
   🎯 Keep m

### Validation of duplicate removal

In [11]:
describe_dataframes(balance_statements_all)


# No duplicates in balance statements or PnL statements
#has_duplicates = B2025_all['ja_kodas'].nunique() < len(B2025_all)

,dataframe_name,rows,columns,total_cells,memory_mb,numeric_columns,categorical_columns,datetime_columns,boolean_columns,unique_dtypes,...,most_unique_column,most_unique_count,least_unique_column,least_unique_count,duplicate_rows,duplicate_row_percentage,total_missing_values,missing_percentage,has_negative_values,has_zero_values
0,df_0,143388,17,2437596,32.55,12,2,3,0,4,...,ja_kodas,143388,standard_id,13,0,0.0,729794,29.94,True,True
1,df_1,137831,17,2343127,31.28,12,2,3,0,4,...,ja_kodas,137831,standard_id,13,0,0.0,704340,30.06,True,True
2,df_2,136576,12,1638912,25.79,7,2,3,0,4,...,ja_kodas,136576,standard_id,11,0,0.0,39846,2.43,True,True
3,df_3,116768,12,1401216,22.05,7,2,3,0,4,...,ja_kodas,116768,standard_id,11,0,0.0,28024,2.00,True,True
4,df_4,103744,12,1244928,19.59,7,2,3,0,4,...,ja_kodas,103744,standard_id,11,0,0.0,20799,1.67,True,True
5,df_5,94664,12,1135968,17.88,7,2,3,0,4,...,ja_kodas,94664,standard_id,11,0,0.0,15596,1.37,True,True



## Final stop: dataframe merge

In [12]:
# Merged financial sheet with new firms in financial year, only matching ja_kodas rows, others are dropped

M_all_matching = merge_similar_dfs(balance_statements_all, pnl_statements_all, 'ja_kodas', how='inner', suffixes=('_balance', '_pnl'), validate='one_to_one', dtype_conversion='auto', verbose=True)

# Merged financial sheet with new firms in financial year, all ja_kodas are kept, matching rows are matched.

M_all_all = merge_similar_dfs(balance_statements_all, pnl_statements_all, 'ja_kodas', how='outer', suffixes=('_balance', '_pnl'), validate='one_to_one', dtype_conversion='auto', verbose=True)

#############################################

# Merged financial sheet only with full financial year data, only matching ja_kodas rows, others are dropped

M_full_matching = merge_similar_dfs(balance_statements_full, pnl_statements_full, 'ja_kodas', how='inner', suffixes=('_balance', '_pnl'), validate='one_to_one', dtype_conversion='auto', verbose=True)

# Merged financial sheet only with full financial year data, all ja_kodas are kept, matching rows are matched

M_full_all = merge_similar_dfs(balance_statements_full, pnl_statements_full, 'ja_kodas', how='outer', suffixes=('_balance', '_pnl'), validate='one_to_one', dtype_conversion='auto', verbose=True)

🔗 MERGE CONFIGURATION:
   📊 Input pairs: 6
   🎯 Merge column: 'ja_kodas'
   🔄 Merge type: inner
   📝 Suffixes: ('_balance', '_pnl') (pandas defaults)
   ✅ Validate: one_to_one
   📈 Indicator: None (no indicator)
   🔄 Duplicate handling: None (no handling)
   🚫 NaN handling: None (no handling)
   🔧 Dtype conversion: auto

--- Pair 0 Merge Diagnostics ---
DF1: 143388 rows, 143388 unique ja_kodas
DF2: 143122 rows, 143122 unique ja_kodas
Common ja_kodas: 142174
Merge type: inner
Expected result rows: 142174
Actual result rows: 142174
Overlapping columns (received suffixes ('_balance', '_pnl')): ['stat_kodas', 'turning_date', 'form_kodas', 'beginning_date', 'reg_date', 'template_id', 'standard_id']
Pair 0: Successfully merged. Shapes: (143388, 17) + (143122, 16) -> (142174, 32)

--- Pair 1 Merge Diagnostics ---
DF1: 137831 rows, 137831 unique ja_kodas
DF2: 137413 rows, 137413 unique ja_kodas
Common ja_kodas: 136680
Merge type: inner
Expected result rows: 136680
Actual result rows: 136680
Ov

### Renaming for saving

In [13]:
# Renaming for dataframes for saving

Merged2025_all_firms_matching = M_all_matching[0]
Merged2024_all_firms_matching = M_all_matching[1]
Merged2023_all_firms_matching = M_all_matching[2]
Merged2022_all_firms_matching = M_all_matching[3]
Merged2021_all_firms_matching = M_all_matching[4]
Merged2020_all_firms_matching = M_all_matching[5]

Merged2025_all_firms_all = M_all_all[0]
Merged2024_all_firms_all = M_all_all[1]
Merged2023_all_firms_all = M_all_all[2]
Merged2022_all_firms_all = M_all_all[3]
Merged2021_all_firms_all = M_all_all[4]
Merged2020_all_firms_all = M_all_all[5]

Merged2025_full_financial_year_matching = M_full_matching[0]
Merged2024_full_financial_year_matching = M_full_matching[1]
Merged2023_full_financial_year_matching = M_full_matching[2]
Merged2022_full_financial_year_matching = M_full_matching[3]
Merged2021_full_financial_year_matching = M_full_matching[4]
Merged2020_full_financial_year_matching = M_full_matching[5]

Merged2025_full_financial_year_all = M_full_all[0]
Merged2024_full_financial_year_all = M_full_all[1]
Merged2023_full_financial_year_all = M_full_all[2]
Merged2022_full_financial_year_all = M_full_all[3]
Merged2021_full_financial_year_all = M_full_all[4]
Merged2020_full_financial_year_all = M_full_all[5]


### Summing financial columns
Since the newer balance and PnL statements have different columns, we need to sum the financial columns to get the values that were given in previous years.

In [14]:

list_for_summing = [Merged2025_all_firms_matching, Merged2024_all_firms_matching, Merged2025_all_firms_all, Merged2024_all_firms_all, Merged2025_full_financial_year_matching, Merged2024_full_financial_year_matching, Merged2025_full_financial_year_all, Merged2024_full_financial_year_all]

# Summing financial columns

# Moketinos sumos ir kiti isipareigojimai
coalesce_columns(list_for_summing, ['MOKĖTINOS SUMOS IR KITI ĮSIPAREIGOJIMAI', 'MOKĖTINOS SUMOS IR ĮSIPAREIGOJIMAI', 'ĮSIPAREIGOJIMAI'], 'MOKĖTINOS SUMOS IR KITI ĮSIPAREIGOJIMAI')

# Nuosavas kapitalas

coalesce_columns(list_for_summing, ['NUOSAVAS KAPITALAS', 'NUOSAVAS KAPITALAS, IŠ VISO:'], 'NUOSAVAS KAPITALAS')

# Ilgalaikis turtas
coalesce_columns(list_for_summing, ['ILGALAIKIS TURTAS', 'ILGALAIKIS MATERIALUSIS TURTAS'], 'ILGALAIKIS TURTAS')

# Bad variable that will conflict with the one which we will create later.
remove_columns(list_for_summing, ['TURTAS'])

# Grynasis pelnas
coalesce_columns(list_for_summing, ['GRYNASIS PELNAS (NUOSTOLIAI)', 'GRYNASIS VEIKLOS REZULTATAS' ], 'GRYNASIS PELNAS (NUOSTOLIAI)')

# Pelnas

coalesce_columns(list_for_summing, ['PELNAS (NUOSTOLIAI) PRIEŠ APMOKESTINIMĄ', 'PELNAS (NUOSTOLIAI) PRIEš APMOKESTINIMą','VEIKLOS REZULTATAS PRIEŠ APMOKESTINIMĄ', 'ATASKAITINIŲ METŲ PELNAS (NUOSTOLIAI)'], 'PELNAS (NUOSTOLIAI) PRIEŠ APMOKESTINIMĄ')

# Pajamos

coalesce_columns(list_for_summing, ['PARDAVIMO PAJAMOS', 'PAJAMOS'], 'PARDAVIMO PAJAMOS')

Coalesced columns ['MOKĖTINOS SUMOS IR ĮSIPAREIGOJIMAI', 'ĮSIPAREIGOJIMAI'] -> 'MOKĖTINOS SUMOS IR KITI ĮSIPAREIGOJIMAI' in DataFrame 0
Coalesced columns ['MOKĖTINOS SUMOS IR ĮSIPAREIGOJIMAI', 'ĮSIPAREIGOJIMAI'] -> 'MOKĖTINOS SUMOS IR KITI ĮSIPAREIGOJIMAI' in DataFrame 1
Coalesced columns ['MOKĖTINOS SUMOS IR ĮSIPAREIGOJIMAI', 'ĮSIPAREIGOJIMAI'] -> 'MOKĖTINOS SUMOS IR KITI ĮSIPAREIGOJIMAI' in DataFrame 2
Coalesced columns ['MOKĖTINOS SUMOS IR ĮSIPAREIGOJIMAI', 'ĮSIPAREIGOJIMAI'] -> 'MOKĖTINOS SUMOS IR KITI ĮSIPAREIGOJIMAI' in DataFrame 3
Coalesced columns ['MOKĖTINOS SUMOS IR ĮSIPAREIGOJIMAI', 'ĮSIPAREIGOJIMAI'] -> 'MOKĖTINOS SUMOS IR KITI ĮSIPAREIGOJIMAI' in DataFrame 4
Coalesced columns ['MOKĖTINOS SUMOS IR ĮSIPAREIGOJIMAI', 'ĮSIPAREIGOJIMAI'] -> 'MOKĖTINOS SUMOS IR KITI ĮSIPAREIGOJIMAI' in DataFrame 5
Coalesced columns ['MOKĖTINOS SUMOS IR ĮSIPAREIGOJIMAI', 'ĮSIPAREIGOJIMAI'] -> 'MOKĖTINOS SUMOS IR KITI ĮSIPAREIGOJIMAI' in DataFrame 6
Coalesced columns ['MOKĖTINOS SUMOS IR ĮSIPAREIG

### Saving the dataframes to csv

In [15]:
# Saving the dataframes to csv

save_df_list_to_csv_auto(M_all_matching, '../../data/interim/joined-balance-and-PnL/only-matching-ja_kodas/with_new_firms')

save_df_list_to_csv_auto(M_all_all, '../../data/interim/joined-balance-and-PnL/all-ja_kodas/with_new_firms')

save_df_list_to_csv_auto(M_full_matching, '../../data/interim/joined-balance-and-PnL/only-matching-ja_kodas/full_financial_year')

save_df_list_to_csv_auto(M_full_all, '../../data/interim/joined-balance-and-PnL/all-ja_kodas/full_financial_year')

['Merged2025_full_financial_year_all',
 'Merged2024_full_financial_year_all',
 'Merged2023_full_financial_year_all',
 'Merged2022_full_financial_year_all',
 'Merged2021_full_financial_year_all',
 'Merged2020_full_financial_year_all']

### Only needed numbers data
We will create versions with only the data, and no explanatory standards and form names.

In [16]:
unneeded_filler_columns = ['template_id_balance', 'standard_id_balance', 'template_id_pnl', 'standard_id_pnl']

remove_columns(M_all_matching, unneeded_filler_columns)
remove_columns(M_all_all, unneeded_filler_columns)
remove_columns(M_full_matching, unneeded_filler_columns)
remove_columns(M_full_all, unneeded_filler_columns)

📊 DataFrame 0:
   ✅ Removed: ['template_id_balance', 'standard_id_balance', 'template_id_pnl', 'standard_id_pnl']
   📋 Remaining columns: 18
📊 DataFrame 1:
   ✅ Removed: ['template_id_balance', 'standard_id_balance', 'template_id_pnl', 'standard_id_pnl']
   📋 Remaining columns: 18
📊 DataFrame 2:
   ✅ Removed: ['template_id_balance', 'standard_id_balance', 'template_id_pnl', 'standard_id_pnl']
   📋 Remaining columns: 18
📊 DataFrame 3:
   ✅ Removed: ['template_id_balance', 'standard_id_balance', 'template_id_pnl', 'standard_id_pnl']
   📋 Remaining columns: 18
📊 DataFrame 4:
   ✅ Removed: ['template_id_balance', 'standard_id_balance', 'template_id_pnl', 'standard_id_pnl']
   📋 Remaining columns: 18
📊 DataFrame 5:
   ✅ Removed: ['template_id_balance', 'standard_id_balance', 'template_id_pnl', 'standard_id_pnl']
   📋 Remaining columns: 18

🎯 FINAL SUMMARY:
   📦 Processed 6 DataFrames
   🗑️  Total columns removed: 24
   ⏭️  Total columns skipped: 0
📊 DataFrame 0:
   ✅ Removed: ['template_id_

[         ja_kodas  form_kodas_balance  stat_kodas_balance  \
 0       110003978               310.0                 0.0   
 1       110004884               310.0                 0.0   
 2       110005648               310.0                 0.0   
 3       110006892               310.0                 0.0   
 4       110008377               310.0                 0.0   
 ...           ...                 ...                 ...   
 132526  306650196               960.0                 0.0   
 132527  306650207               960.0                 0.0   
 132528  306650214               570.0                 0.0   
 132529  306650221               570.0                 0.0   
 132530  306650239               310.0                 0.0   
 
        beginning_date_balance turning_date_balance reg_date_balance  \
 0                  2024-01-01           2024-12-31       2025-05-28   
 1                  2024-01-01           2024-12-31       2025-05-28   
 2                  2024-01-01        

### Adding new columns

### Standard and form code merges
We will merge the stat and form code columns to get a single column with the standard and form code for each row, since they are identical in different columns.

In [17]:
# stat_kodas
coalesce_columns(M_all_matching, ['stat_kodas_balance','stat_kodas_pnl'], 'stat_kodas')
coalesce_columns(M_all_all, ['stat_kodas_balance','stat_kodas_pnl'], 'stat_kodas')
coalesce_columns(M_full_matching, ['stat_kodas_balance','stat_kodas_pnl'], 'stat_kodas')
coalesce_columns(M_full_all, ['stat_kodas_balance','stat_kodas_pnl'], 'stat_kodas')

# form_kodas
coalesce_columns(M_all_matching, ['form_kodas_balance', 'form_kodas_pnl'], 'form_kodas')
coalesce_columns(M_all_all, ['form_kodas_balance', 'form_kodas_pnl'], 'form_kodas')
coalesce_columns(M_full_matching, ['form_kodas_balance', 'form_kodas_pnl'], 'form_kodas')
coalesce_columns(M_full_all, ['form_kodas_balance', 'form_kodas_pnl'], 'form_kodas')

Coalesced columns ['stat_kodas_balance', 'stat_kodas_pnl'] -> 'stat_kodas' in DataFrame 0
Coalesced columns ['stat_kodas_balance', 'stat_kodas_pnl'] -> 'stat_kodas' in DataFrame 1
Coalesced columns ['stat_kodas_balance', 'stat_kodas_pnl'] -> 'stat_kodas' in DataFrame 2
Coalesced columns ['stat_kodas_balance', 'stat_kodas_pnl'] -> 'stat_kodas' in DataFrame 3
Coalesced columns ['stat_kodas_balance', 'stat_kodas_pnl'] -> 'stat_kodas' in DataFrame 4
Coalesced columns ['stat_kodas_balance', 'stat_kodas_pnl'] -> 'stat_kodas' in DataFrame 5
Coalesced columns ['stat_kodas_balance', 'stat_kodas_pnl'] -> 'stat_kodas' in DataFrame 0
Coalesced columns ['stat_kodas_balance', 'stat_kodas_pnl'] -> 'stat_kodas' in DataFrame 1
Coalesced columns ['stat_kodas_balance', 'stat_kodas_pnl'] -> 'stat_kodas' in DataFrame 2
Coalesced columns ['stat_kodas_balance', 'stat_kodas_pnl'] -> 'stat_kodas' in DataFrame 3
Coalesced columns ['stat_kodas_balance', 'stat_kodas_pnl'] -> 'stat_kodas' in DataFrame 4
Coalesced 

### Saving dfs with less filler to csv

In [18]:
# Saving the dataframes to csv

save_df_list_to_csv_auto(M_all_matching, '../../data/interim/joined-balance-and-PnL-no-filler/only-matching-ja_kodas/with_new_firms')

save_df_list_to_csv_auto(M_all_all, '../../data/interim/joined-balance-and-PnL-no-filler/all-ja_kodas/with_new_firms')

save_df_list_to_csv_auto(M_full_matching, '../../data/interim/joined-balance-and-PnL-no-filler/only-matching-ja_kodas/full_financial_year')

save_df_list_to_csv_auto(M_full_all, '../../data/interim/joined-balance-and-PnL-no-filler/all-ja_kodas/full_financial_year')

['Merged2025_full_financial_year_all',
 'Merged2024_full_financial_year_all',
 'Merged2023_full_financial_year_all',
 'Merged2022_full_financial_year_all',
 'Merged2021_full_financial_year_all',
 'Merged2020_full_financial_year_all']